In [1]:
import numpy as np
import re
import os
import sys
import subprocess
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [41]:
import os
for a in os.listdir('../results')[:]:
    print(a)
    with open('../results/{}'.format(a)) as f:
        b = f.readlines()
        b = list(map(lambda x:x.strip().split(','),b))
        b = list(map(lambda x: list(filter(len,x)),b))
        
    with open('../results/{}'.format(a),'w') as f:
        f.writelines(list(map(lambda x:','.join(x)+'\n',b)))

A3QS7M912JGKFD.txt
A3L60SOWLYEDN2.txt
A28A3HF3LSEIDT.txt
A3AKZJNAUP8LSA.txt
A1IHI23KH87K5W.txt
A3QI1RV4HQ9MOC.txt
A1FHS282JP487T.txt
AISFU4P55HYEH.txt
A2BO8M77CS3SGZ.txt
A257UIPAZR9NTN.txt
A2LEY4X3LD8G26.txt
A3VOMP0WOJTB4I.txt
A1IOMFFEKCWOIT.txt
A2YHSU2IJZ9J14.txt
A299J4PKHAEU9H.txt
A2J237J8KM3OCS.txt
A1XO6ONCCTBMKW.txt
A3UUH3632AI3ZX.txt
A2BAQ26SMQQEUG.txt
A341XKSRZ58FJK.txt
A2QD9PJUKW7PKK.txt
APSN3KV49VLKX.txt
A1CF6U3GF7DZEJ.txt
ASWZO1WY1UHEP.txt
A3TUMZ954ORSUC.txt
A1IU5OP7BBZHZ7.txt
AA7BTDZQNHW5.txt
A5YNMUFWH5Z7K.txt
A37M4OVE9R80G.txt
A2R0YYUAWNT7UD.txt
A2PYXFVGNJPPX0.txt
A10NF5TK6IFNX6.txt
a2h4hydu6ffamq.txt
A1DVKS3R9SLQ1H.txt
A26399B1QZ7XJJ.txt
A5LYLHG880ABE.txt
A26RPQDD0RQEHL.txt
ABM77ZQWCHPNX.txt


In [5]:
subprocess.run('mv ../rejected_results/* ../results',shell=True)
#Sample filter function
def filter_single_video(video_times, rating_times, video_order, scores,attentions):
    #First check if user watching alines with video length
    #0.5 sec tolerance for black screen at the end
    ret = 0
    bad = 0
    violate = 0
    a = 0
    for k in video_times[:-1]:
        if k < 7000:
            a += 1
    if video_times[-1] < 4000:
        a +=1
    if scores[-1] != max(scores):
        a+=1
    if attentions[0] != 1:
        a+=1
    #if scores[3] > scores[2] or scores[1] > scores[0]:
    #    a+=1

    if a >0:
        return 1
    return 0 #We don't move this user to rejected folder

#Parse data from user result file 
def parse_results(lines):
    #video_times = list(map(int,lines[2].strip().split(','))) #read times spent on each video  
    #rating_times = list(map(int,lines[3].strip().split(','))) #read times spent on each rating  
    video_times = []; rating_times = []
    video_order = list(map(int,lines[1].strip().split(','))) #read the video order seen by the surveyee
    scores = list(map(int,lines[0].strip().split(','))) #read scores
    attentions = list(map(int,lines[-2].strip().split(','))) #attention checks    
    video_times =  list(map(int,lines[2].strip().split(','))) 
    return video_times, rating_times, video_order, scores, attentions


#input from the cmd line script
result_path = '../results/'
#white_list = '../good/'


move = False
result_files = os.listdir(result_path)
#white_list_files = os.listdir(white_list)

total = 0
approved = 0
reject = 0
pending = 0
for result_file in result_files:
    #filter a single result
    total +=1
    '''
    if result_file in white_list_files:
        approved +=1
        continue
    '''
    result = result_path + result_file
    #print(result)
    with open(result, "r") as fp:
        lines = fp.readlines()
        move = False

        #insert customized filter here 
        move = filter_single_video(*parse_results(lines))
        if move == 1:
            reject +=1
            print(result_file)
            os.system("mv {} ../rejected_results/".format(result))
        else:
            approved +=1

print('reject:',reject)
print('approved:',approved)
print('rejection ratio:', reject/(reject+approved))


A3AKZJNAUP8LSA.txt
A3QI1RV4HQ9MOC.txt
A1CF6U3GF7DZEJ.txt
A3TUMZ954ORSUC.txt
A37M4OVE9R80G.txt
A2PYXFVGNJPPX0.txt
a2h4hydu6ffamq.txt
A1DVKS3R9SLQ1H.txt
A5LYLHG880ABE.txt
reject: 9
approved: 29
rejection ratio: 0.23684210526315788


In [43]:
subprocess.run('./get_results.sh',shell=True)

CompletedProcess(args='./get_results.sh', returncode=0)

In [47]:
subprocess.run('python3 ./create_csv.py',shell=True, cwd="/tank/yiyangou/web_QoE2")

CompletedProcess(args='python3 ./create_csv.py', returncode=0)

In [2]:
def to_int(df, names):
    for n in names:
        df[n] = pd.to_numeric(df[n])
    return df
def scale(a):
    return (a - a.min())/(a.max()-a.min())

In [5]:
df = pd.read_csv('../sum.csv',header=None)
print(list(df[6]))
columns = ['scores','video_order','time','reason']
#df = df[~df[6].isin(c)]
df = df[[0,1,2,10]]
df.columns = columns
vid = ['score' + str(i+1) for i in range(3)]
df[vid]=df.scores.str.split(',',expand=True)
df =to_int(df,vid)
order = ['order' + str(i+1) for i in range(3)]
df[order]=df.video_order.str.split(',',expand=True)
df =to_int(df,order)
df = df[[*vid,*order,'reason']]

['A3QS7M912JGKFD', 'A3L60SOWLYEDN2', 'A28A3HF3LSEIDT', 'A1IHI23KH87K5W', 'A1FHS282JP487T', 'AISFU4P55HYEH', 'A2BO8M77CS3SGZ', 'A257UIPAZR9NTN', 'A2LEY4X3LD8G26', 'A3VOMP0WOJTB4I', 'A1IOMFFEKCWOIT', 'A2YHSU2IJZ9J14', 'A299J4PKHAEU9H', 'A2J237J8KM3OCS', 'A1XO6ONCCTBMKW', 'A3UUH3632AI3ZX', 'A2BAQ26SMQQEUG', 'A341XKSRZ58FJK', 'A2QD9PJUKW7PKK', 'APSN3KV49VLKX', 'ASWZO1WY1UHEP', 'A1IU5OP7BBZHZ7', 'AA7BTDZQNHW5', 'A5YNMUFWH5Z7K', 'A2R0YYUAWNT7UD', 'A10NF5TK6IFNX6', 'A26399B1QZ7XJJ', 'A26RPQDD0RQEHL', 'ABM77ZQWCHPNX']


In [232]:
def consistency(df):
    return ((df['score2'] > df['score1']) & (df['score4'] > df['score3'])) | \
    ((df['score2'] < df['score1']) & (df['score4'] < df['score3'])) | \
((df['score2'] == df['score1']) & (df['score4'] == df['score3']))
    

In [38]:
a = set(['A2QD9PJUKW7PKK', 'AM0JKZVOEOTMA', 'A1IU5OP7BBZHZ7', 'A3KU8DC7HJOGCM', 'A3A8P4UR9A0DWQ', 'AISFU4P55HYEH', 'A1DVKS3R9SLQ1H', 'A28A3HF3LSEIDT', 'A3AKZJNAUP8LSA', 'A1IHI23KH87K5W', 'A9EVF0SO2IUJ0', 'AA7BTDZQNHW5', 'A5LYLHG880ABE', 'A2SSYBUT9ZM0G9', 'A2PBRI6BOYDN94', 'A2BO8M77CS3SGZ', 'ASWZO1WY1UHEP', 'A1FPCIKO68OQ63', 'A2YHSU2IJZ9J14', 'A1IOMFFEKCWOIT', 'A3VOMP0WOJTB4I', 'A1XO6ONCCTBMKW', 'A2J237J8KM3OCS'])
b = set(['A3QS7M912JGKFD', 'A3L60SOWLYEDN2', 'A28A3HF3LSEIDT', 'A1IHI23KH87K5W', 'AISFU4P55HYEH', 'A2BO8M77CS3SGZ', 'A3VOMP0WOJTB4I', 'A1IOMFFEKCWOIT', 'A2YHSU2IJZ9J14', 'A299J4PKHAEU9H', 'A2J237J8KM3OCS', 'A1XO6ONCCTBMKW', 'A2BAQ26SMQQEUG', 'A341XKSRZ58FJK', 'A2QD9PJUKW7PKK', 'APSN3KV49VLKX', 'ASWZO1WY1UHEP', 'A1IU5OP7BBZHZ7', 'AA7BTDZQNHW5', 'A5YNMUFWH5Z7K', 'A37M4OVE9R80G', 'A10NF5TK6IFNX6', 'A1DVKS3R9SLQ1H', 'A26399B1QZ7XJJ', 'A26RPQDD0RQEHL', 'ABM77ZQWCHPNX'])
c = a.intersection(b)

In [7]:
sum(df['score1']>df['score2']),sum(df['score1']<df['score2']),sum(df['score1']==df['score2'])

(9, 4, 16)

In [4]:
sum(df['score1']>df['score2']),sum(df['score1']<df['score2']),sum(df['score1']==df['score2'])

(9, 4, 16)

In [196]:
len(df2)

6

In [76]:
print(df.sem())

df.mean()

score1    0.183468
score2    0.198503
score3    0.186737
score4    0.195431
score5    0.099446
dtype: float64


score1    2.930233e+00
score2    3.139535e+00
score3    2.976744e+00
score4    3.023256e+00
score5    4.837209e+00
order1    3.126545e+40
order2    5.658892e+40
order3    9.584284e+40
order4    7.470073e+40
order5    1.291990e+41
dtype: float64